In [1]:
from parser import bus_data_df, load_data_df, generator_data_df, fixed_shunt_data_df, branch_data_df, trafo_data_df, switched_shunt_data_df
import pandas as pd
import pandapower as pp
import numpy as np
import os

raw_file = '/Users/tomasvazquez/Desktop/Facultad/Tesis/Datos/DORAA_una_carga.raw'
raw_file = '/Users/tomasvazquez/Desktop/Facultad/Tesis/Datos/DORAA_una_carga_un_gen.raw'

## Cargamos datos del .raw

In [2]:
bus_data = bus_data_df(raw_file)
load_data = load_data_df(raw_file)
generator_data = generator_data_df(raw_file)
fixed_shunt_data = fixed_shunt_data_df(raw_file)
branch_data = branch_data_df(raw_file)
trafo_data = trafo_data_df(raw_file)
switched_shunt_data = switched_shunt_data_df(raw_file)

In [5]:
uy_net = pp.create_empty_network(sn_mva = 100)

### Cargamos bus data

In [6]:
for bus in bus_data.index:
    pp.create_bus(uy_net, vn_kv=bus_data.loc[bus, 'BASKV'],
                            name = bus,
                            index=bus_data.loc[bus, 'I'],
                            zone = bus_data.loc[bus, 'ZONE'],
                            in_service = bus_data.loc[bus, 'IDE'] != 4,
                            max_vm_pu = bus_data.loc[bus, 'NVHI'],
                            min_vm_pu = bus_data.loc[bus, 'NVLO']
                            )

### Creamos load data

In [7]:
for bus in load_data.index:
    pp.create_load(uy_net, bus=int(bus),
                    p_mw=load_data.loc[bus, 'PL'],
                    q_mvar=load_data.loc[bus, 'QL'],
                    name = None,
                    index=None,
                    in_service = True,
                    controllable=False
                    )

### Creamos fixed shunt data

In [8]:
for idx in fixed_shunt_data.index:
    pp.create_shunt(uy_net, bus=int(fixed_shunt_data.loc[idx, 'I']),
                    q_mvar=-fixed_shunt_data.loc[idx, 'BL'],
                    p_mw=fixed_shunt_data.loc[idx, 'GL'],
                    )

### Creamos generator data

In [9]:
for bus in generator_data.index:
    if bus != '4000':
        pp.create.create_gen(uy_net, bus=int(bus),
                            p_mw=generator_data.loc[bus, 'PG'],
                            # vm_pu=generator_data.loc[bus, 'VS'],
                            vm_pu=1.0,
                            name = None,
                            index=None,
                            in_service = True,
                            controllable=True,
                            max_p_mw = generator_data.loc[bus, 'PT'],
                            min_p_mw = generator_data.loc[bus, 'PB'],
                            max_q_mvar = generator_data.loc[bus, 'QT'],
                            min_q_mvar = generator_data.loc[bus, 'QB'],
                            )

### Creamos branch data

In [10]:
for idx in branch_data.index:
    V_base = bus_data.loc[bus_data['I'] == int(branch_data.loc[idx]['I'])]['BASKV'].values[0]
    pp.create_line_from_parameters(uy_net, from_bus=int(branch_data.loc[idx, 'I']),
                    to_bus=int(branch_data.loc[idx, 'J']),
                    length_km=1,
                    r_ohm_per_km=branch_data.loc[idx, 'R']*V_base**2/100,
                    x_ohm_per_km=branch_data.loc[idx, 'X']*V_base**2/100,
                    c_nf_per_km=(branch_data.loc[idx, 'B'] + (branch_data.loc[idx, 'BI'] + branch_data.loc[idx, 'BJ'])/2) / (2*np.pi*50*1e-9)*(100/V_base**2),
                    g_us_per_km= (branch_data.loc[idx, 'GI'] + branch_data.loc[idx, 'GJ']) * (100/V_base**2) * 1e6,
                    max_i_ka=float(branch_data.loc[idx, 'RATE1']) / (np.sqrt(3) * V_base),
                    max_loading_percent = 100,
                    name=None,
                    index=None,
                    in_service=True,
                    )

### Creando transfomer data

In [11]:
for idx in trafo_data.index:
    pp.create_transformer_from_parameters(uy_net,
                                    hv_bus=int(trafo_data.loc[idx, 'I']),
                                    lv_bus=int(trafo_data.loc[idx, 'J']),
                                    sn_mva=trafo_data.loc[idx, 'RATE1-1'],
                                    vn_hv_kv = 500,
                                    vn_lv_kv = 150,
                                    vkr_percent = trafo_data.loc[idx, 'R1-2'] * 100,
                                    vk_percent = ((trafo_data.loc[idx, 'R1-2']**2 + trafo_data.loc[idx, 'X1-2']**2)**0.5) * 100,
                                    pfe_kw = 0.0,
                                    i0_percent = 0.0,
                                    in_service = True,
                                    vector_group = trafo_data.loc[idx, 'VECGRP'],
                                    max_loading_percent = 100,
                                   )

### Creando external grid

In [12]:
pp.create.create_ext_grid(uy_net,
                            bus = 4000,
                            vm_pu = 1.0,
                            va_degree = 0,
                            min_p_mw = None,
                            max_p_mw = None,
                            min_q_mvar = -10,
                            max_q_mvar = 10
                          )

0

### Agregamos los switched shunts como static generators

In [13]:
for idx in switched_shunt_data.index:
    if switched_shunt_data.loc[idx, 'B1']<=0:
        pp.create_sgen(uy_net,
                    bus = int(switched_shunt_data.loc[idx, 'I']),
                        p_mw= 0,
                        q_mvar = switched_shunt_data.loc[idx, 'BINIT'],
                        max_p_mw= 0,
                        min_p_mw = 0,
                        min_q_mvar= switched_shunt_data.loc[idx, 'N1'] * switched_shunt_data.loc[idx, 'B1'],
                        max_q_mvar= 0,
                        controllable=True)

    if switched_shunt_data.loc[idx, 'B1']>0:
        pp.create_sgen(uy_net,
                    bus = int(switched_shunt_data.loc[idx, 'I']),
                        p_mw= 0,
                        q_mvar = switched_shunt_data.loc[idx, 'BINIT'],
                        max_p_mw= 0,
                        min_p_mw = 0,
                        min_q_mvar= 0,
                        max_q_mvar= switched_shunt_data.loc[idx, 'N1'] * switched_shunt_data.loc[idx, 'B1'],
                        controllable=True)

### Opcional: pasar eólico y solar a sgen

In [14]:
buses_solar = [92040,92030,92020,92070,92640,92020, 92060, 92000]
buses_eolico = [92710,92091,92760,92440,92611,92700,92590,92061,92610,92580,92790,92485,92420,92620,92910,92524,92411,92490] #90160

for idx, g in uy_net.gen.iterrows():
    if g.bus in buses_solar:
        pp.create_sgen(uy_net, g.bus,  g.p_mw,controllable=False)
        uy_net.gen.drop(idx,inplace=True)
    if g.bus in buses_eolico:
        pp.create_sgen(uy_net, g.bus,  g.p_mw,controllable=False)
        uy_net.gen.drop(idx,inplace=True)

### Guardamos la red

In [15]:
pp.to_pickle(uy_net, '/Users/tomasvazquez/Desktop/Facultad/Tesis/Datos/uy_pp_net_v13_(sin_eolico_ni_solar).p')  # absolute path

### Visualización de la red

In [18]:
uy_net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent
0,None,None,101,92740,1.0,0.149565,0.245700,487.226332,0.0,0.431088,1.0,1,None,True,100.0
1,None,None,101,92740,1.0,0.149565,0.245700,487.226332,0.0,0.431088,1.0,1,None,True,100.0
2,None,None,4000,4004,1.0,4.625000,43.175000,2171.255396,0.0,1.199734,1.0,1,None,True,100.0
3,None,None,4000,90000,1.0,0.100000,1.025000,49.529018,0.0,1.199734,1.0,1,None,True,100.0
4,None,None,4004,90080,1.0,0.700000,6.475000,314.235520,0.0,1.199734,1.0,1,None,True,100.0
5,None,None,90000,90080,1.0,4.275000,40.150000,1947.101574,0.0,1.199734,1.0,1,None,True,100.0
6,None,None,90080,90500,1.0,2.286710,21.414125,1038.772483,0.0,2.167373,1.0,1,None,True,100.0
7,None,None,90080,90500,1.0,2.360285,22.103125,1072.195021,0.0,1.199999,1.0,1,None,True,100.0
8,None,None,90100,90180,1.0,1.525000,15.420000,774.740798,0.0,1.611962,1.0,1,None,True,100.0
9,None,None,90100,90200,1.0,0.290593,2.721275,132.009476,0.0,2.167373,1.0,1,None,True,100.0


In [17]:
pp.plotting.simple_plot(uy_net)

No or insufficient geodata available --> Creating artificial coordinates. This may take some time


ImportError: Some pandapower functionality use modules outside the setup.py requirements: build_igraph_from_pp() requires 'igraph'. 
igraph could not be imported.
To install all pandapower dependencies, pip install pandapower["all"] can be used.

In [11]:
uy_net.ext_grid

,name,bus,vm_pu,va_degree,slack_weight,in_service


In [22]:
arg = ['Importación Argentina CAMMESA modalidad devolución',
       'Importación Argentina CAMMESA Gasoil',
       'Importación Argentina CAMMESA Fueloil',
       'Importación Argentina Cemsa 1',
       'Importación Argentina Cemsa 2',
       'Importación Argentina CAMMESA Contingente',
       'IMP CONTINGENTE CAMMESA BLQ 1',
       'IMP CONTINGENTE CAMMESA BLQ 2',
       'IMP CONTINGENTE CAMMESA BLQ 3',
       'IMP CONTINGENTE CAMMESA BLQ 4',
       'IMP CONTINGENTE CAMMESA BLQ 5',
       'Importación Argentina CAMESSA Emergencia',
       'Exportación Argentina VECODESA-CAMMESA',
       'Exportación Argentina CEOSA-CAMMESA',
       'Exportación Argentina Bloque 2',
        'Exportación Argentina Bloque 1',
        'Exportación Argentina VECODESA-CAMMESA',
        'Exportación Argentina CEOSA-CAMMESA',
        'Exportación Argentina Bloque 2',
        'Exportación Argentina Bloque 1',
        'EXP CONTINGENTE CAMMESA BLQ 3',
        'Exportación Argentina Térmica modalidad Contingente',
        'EXP CONTINGENTE CAMMESA BLQ 5',
        'EXP CONTINGENTE CAMMESA BLQ 4',
        'EXP CONTINGENTE CAMMESA BLQ 6',
        'EXP CONTINGENTE CAMMESA BLQ 6',
        'EXP CONTINGENTE CAMMESA BLQ 0',
        'Exportacion CAMMESA sustitucion']

bra = [
# 'Imp. Energia devolución de Brasil',
# 'Importación Brasil Contrato TRADENER',
# 'Exportación Brasil Bloque 2 ENEL',
# 'Exportación Brasil Bloque 3 ENEL',
# 'Exportación Brasil Bloque 4 ENEL',
# 'Contrato Genérico (NO USAR)',
# 'Exportación Brasil Bloque 4 ELETROBRAS',
# 'Exportación Brasil Bloque 3 ELETROBRAS',
# 'Exportación Brasil Bloque 1 ELETROBRAS',
# 'Exportación Devolución a Brasil Melo',
'Exp Brasil BGT Pactual Bloque 2',
# 'Exportación Brasil Bloque 2 ELETROBRAS',
'Exp Brasil BGT Pactual Bloque 1',
# 'Exportación Brasil Bloque 5 ENEL',
# 'Exportación Brasil Bloque 5 ELETROBRAS',
# 'Exportacion Devolucion a Brasil Rivera',
# 'Exportación Brasil Ensayos Interconexión',
# 'Importación Brasil Ensayos Interconexión',
# 'Exportación Emergencia a Brasil'
# 'Impo Br TRADENER Bloque 1',
# 'IMP CONT. BOLT MELO BLQ 2',
# 'IMP CONT. BOLT MELO BLQ 4',
# 'Impo Br TRADENER Bloque 4',
# 'Impo Br TRADENER Bloque 3',
# 'Impo Br TRADENER Bloque 2',
# 'IMP CONT. BOLT MELO BLQ 1',
# 'IMP CONT. BOLT MELO BLQ 3']

bra_melo = [
'Imp. Energia devolución de Brasil',
'Exportación Devolución a Brasil Melo',
'IMP CONT. BOLT MELO BLQ 1',
'IMP CONT. BOLT MELO BLQ 3',
'IMP CONT. BOLT MELO BLQ 2',
'IMP CONT. BOLT MELO BLQ 4',
'Exportación Brasil Bloque 2 ENEL',
'Exportación Brasil Bloque 3 ENEL',
'Exportación Brasil Bloque 4 ENEL',
'Exportación Brasil Bloque 5 ENEL', # Puede ser riv
'Exportación Brasil Bloque 5 ELETROBRAS', # Puede ser riv
'Exportación Brasil Bloque 2 ELETROBRAS',
'Exportación Brasil Bloque 4 ELETROBRAS',
'Exportación Brasil Bloque 3 ELETROBRAS',
'Exportación Brasil Bloque 1 ELETROBRAS',
'Impo Br TRADENER Bloque 4',
'Impo Br TRADENER Bloque 3',
'Impo Br TRADENER Bloque 2',
'Impo Br TRADENER Bloque 1',
'Exportación Emergencia a Brasil'
'Exportación Brasil Ensayos Interconexión',
'Importación Brasil Ensayos Interconexión',
]

bra_rivera = [

'Importación Brasil Contrato TRADENER',
'Exportacion Devolucion a Brasil Rivera',

]

SyntaxError: invalid syntax (1162300994.py, line 59)